In [ ]:
from estival.model import BayesianCompartmentalModel
from estival import targets as est

import tbh.runner_tools as rt
from tbh.model import get_tb_model

In [ ]:
params, priors, tv_params = rt.get_parameters_and_priors()
model = get_tb_model(rt.DEFAULT_MODEL_CONFIG, tv_params)
bcm = BayesianCompartmentalModel(model, params, priors, rt.targets)

In [ ]:
import nevergrad as ng
from estival.wrappers.nevergrad import optimize_model

In [ ]:
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class)
rec = orunner.minimize(2000)

In [ ]:
mle_params = rec.value[1]
mle_params
res = bcm.run(mle_params)

In [ ]:
mle_params

In [ ]:
import matplotlib.pyplot as plt
import math

# Number of targets
n_targets = len(bcm.targets)
n_cols = 3
n_rows = math.ceil(n_targets / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axes = axes.flatten()  # make it easy to index

for i, (t_name, t) in enumerate(bcm.targets.items()):
    ax = axes[i]
    
    t_data = t.data
    series = res.derived_outputs[t_name].loc[2010:2025]
    
    # Plot main line
    series.plot(ax=ax, title=t_name)
    
    # Plot single-point data as dot
    t_data.plot(ax=ax, style="o")
    
    # Set y-limits
    ax.set_ylim(bottom=0)
    ymax = series.max()
    ax.set_ylim(top=max([1.3 * ymax, t_data.max()]))
    ax.set_xlabel("Year")
    ax.set_ylabel("Value")

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
year = 2025
deno = res.derived_outputs['tb_prevalence'].loc[year]
for state in ["subclin_lowinf", "clin_lowinf", "subclin_inf", "clin_inf"]:
    prev = res.derived_outputs[f"prev_{state}"].loc[year]
    print(f"% {state}: {round(100 * prev / deno)}")

In [ ]:
year = 2025
deno = res.derived_outputs['tbi_prevalence'].loc[year]
for state in ["incipient", "contained", "cleared"]:
    prev = res.derived_outputs[f"prev_{state}"].loc[year]
    print(f"% {state}: {round(100 * prev / deno)}")